<a href="https://colab.research.google.com/github/KseniyaEremeychik/algorithms-massive-data-unimi/blob/main/Kseniya_Yerameichyk_MBA_Exam_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('Market Basket Analysis').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark
import pyspark
type(spark)

spark.sparkContext.setSystemProperty('spark.executor.memory', '16g')
sc = spark.sparkContext
spark

In [ ]:
from google.colab import userdata
import os

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024 -f job_skills.csv
!unzip "job_skills.csv.zip"

Dataset URL: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024
License(s): ODC Attribution License (ODC-By)
 99% 195M/197M [00:01<00:00, 160MB/s]
100% 197M/197M [00:01<00:00, 152MB/s]
Archive:  job_skills.csv.zip
  inflating: job_skills.csv          


### **EDA**

In [ ]:
df = spark.read.options(header=True).csv('job_skills.csv')
df.show()

+--------------------+--------------------+
|            job_link|          job_skills|
+--------------------+--------------------+
|https://www.linke...|Building Custodia...|
|https://www.linke...|Customer service,...|
|https://www.linke...|Applied Behavior ...|
|https://www.linke...|Electrical Engine...|
|https://www.linke...|Electrical Assemb...|
|https://www.linke...|Access Control, V...|
|https://www.linke...|Consultation, Sup...|
|https://www.linke...|Veterinary Recept...|
|https://www.linke...|Optical Inspectio...|
|https://www.linke...|HVAC, troubleshoo...|
|https://www.linke...|Host/Server Assis...|
|https://www.linke...|Apartment mainten...|
|https://www.linke...|Fiber Optic Cable...|
|https://www.linke...|CT Technologist, ...|
|https://ca.linked...|SAP, DRMIS, Data ...|
|https://www.linke...|Debt and equity o...|
|https://ca.linked...|Biomedical Engine...|
|https://www.linke...|Laboratory Techni...|
|https://www.linke...|Program Managemen...|
|https://www.linke...|Hiring, Tr

In [ ]:
df = df.drop("job_link");
df.show();

+--------------------+
|          job_skills|
+--------------------+
|Building Custodia...|
|Customer service,...|
|Applied Behavior ...|
|Electrical Engine...|
|Electrical Assemb...|
|Access Control, V...|
|Consultation, Sup...|
|Veterinary Recept...|
|Optical Inspectio...|
|HVAC, troubleshoo...|
|Host/Server Assis...|
|Apartment mainten...|
|Fiber Optic Cable...|
|CT Technologist, ...|
|SAP, DRMIS, Data ...|
|Debt and equity o...|
|Biomedical Engine...|
|Laboratory Techni...|
|Program Managemen...|
|Hiring, Training,...|
+--------------------+
only showing top 20 rows



In [ ]:
rows = df.count();
print("The initial number of rows in the dataset: ", rows);

df = df.dropna();
rows_without_na = df.count();
print("The number of rows in the dataset after removing the missing values: ", rows_without_na);

The initial number of rows in the dataset:  1296381
The number of rows in the dataset after removing the missing values:  1294374


In [ ]:
sample_size = 0.002;
df_sample = df.sample(withReplacement=False, fraction=sample_size, seed=123);

print("The size of the sample: ", df_sample.count());

The size of the sample:  2521


In [ ]:
rdd = df_sample.rdd;
rdd.cache();

num_cores = sc.defaultParallelism
new_num_partitions = num_cores * 2
rdd = rdd.repartition(new_num_partitions)
num_partitions = rdd.getNumPartitions();

print("The number of partitions: ", num_partitions);

The number of partitions:  4


In [ ]:
baskets = rdd.map(lambda row: row.job_skills.split(', '))

In [ ]:
print(baskets.first())

['Clinical Procedures', 'Cardiac Cath Lab', 'Imaging Catheters', 'Diagnostic', 'Atherectomy', 'Medical Devices', 'Balloon Dilatation Catheters', 'Technical Support', 'Patient Care', 'Product Troubleshooting', 'Product Training', 'Product Demonstrations', 'Customer Education', 'Case Coverage', 'Sales Support', 'Sales Goals', 'Vascular Products', 'Coronary Portfolio', 'Interventional Radiology', 'Operating Room', 'Customer Interaction', 'Bachelors Degree', 'Patient Interaction', 'Relevant Technical Certification', 'Travel']


In [ ]:
size_rdd = baskets.map(lambda row: len(row))

max_size = size_rdd.max()
print("The size of the biggest skills set: ", max_size)

min_size = size_rdd.min()
print("The size of the smallest skills set: ", min_size)

total_size = size_rdd.sum()
count = size_rdd.count()
average_size = total_size / count
print("The average size of the skills set: ", round(average_size))

The size of the biggest skills set:  144
The size of the smallest skills set:  1
The average size of the skills set:  21


In [ ]:
global_threshold = df.count() * 0.01;
sample_threshold = round(global_threshold * sample_size);

print("The threshold value: ", sample_threshold)

The threshold value:  26


### **Data Preprocessing**

In [ ]:
unique_skills = baskets.flatMap(lambda x: x).distinct()
print("The number of unique skills: ", unique_skills.count())

The number of unique skills:  26832


In [ ]:
skills_hash = list(enumerate(unique_skills.collect()))
skill_to_id = {skill: id for id, skill in skills_hash}
id_to_skill = {id: skill for id, skill in skills_hash}

In [ ]:
print(list(skill_to_id.items())[:5])
print(list(id_to_skill.items())[:5])

[('Imaging Catheters', 0), ('Technical Support', 1), ('Product Troubleshooting', 2), ('Product Training', 3), ('Product Demonstrations', 4)]
[(0, 'Imaging Catheters'), (1, 'Technical Support'), (2, 'Product Troubleshooting'), (3, 'Product Training'), (4, 'Product Demonstrations')]


In [ ]:
hashed_baskets = baskets.map(lambda basket: [skill_to_id[skill] for skill in basket])

In [ ]:
hashed_baskets.first()[:10]

[6640, 6641, 0, 20145, 20146, 13330, 13331, 1, 13332, 2]

### **Apriori Algorithm**

In [ ]:
from itertools import combinations

In [ ]:
def apriori_algorithm(baskets, support, hash_dict):
    print("Finding frequent singletons ...")

    frequent_rdd = baskets.flatMap(lambda basket: [(skill, 1) for skill in basket]) \
                          .reduceByKey(lambda x, y: x+y) \
                          .filter(lambda x: x[1] > support)

    frequent_rdd_count = frequent_rdd.count()

    if frequent_rdd_count == 0:
        print("There is no frequent singletons. Try to rerun with the lower support value")
        return

    print("The amount of frequent singletons:", frequent_rdd_count)

    most_freq_singleton = frequent_rdd.max(lambda x: x[1])
    most_freq_singleton_name = id_to_skill[most_freq_singleton[0]]
    print("The most frequent singleton skill:", most_freq_singleton_name)
    print("The most frequent singleton support value:", most_freq_singleton[1])
    print()

    frequent_set = set(frequent_rdd.map(lambda x: (x[0],)).collect())

    itemset_size = 2
    while frequent_rdd_count != 0:
        if itemset_size == 2:
            print("Finding frequent pairs ...")
        elif itemset_size == 3:
            print("Finding frequent triplets ...")
        else:
            print("Finding frequent itemsets of size", itemset_size, "...")

        frequent_rdd = baskets.flatMap(lambda basket: [(itemset, 1) for itemset in combinations(sorted(basket), itemset_size)
                                                      if all(x in frequent_set for x in combinations(itemset, itemset_size-1))]) \
                              .reduceByKey(lambda x, y: x+y) \
                              .filter(lambda x: x[1] > support)

        frequent_set = set(frequent_rdd.map(lambda x: x[0]).collect())
        frequent_rdd_count = frequent_rdd.count();

        if frequent_rdd_count == 0:
            print("There are no frequent itemsets of the size", itemset_size)
            print("According to the monotonicity assumption there are no frequent itemsets of the bigger size")
            print()
            break
        else:
            if itemset_size == 2:
                print("The amount of frequent pairs:", frequent_rdd_count)
            elif itemset_size == 3:
                print("The amount of frequent triplets:", frequent_rdd_count)
            else:
                print("The amount of frequent itemsets of size", itemset_size, ":", frequent_rdd_count)
            itemset_size += 1

        most_freq_itemset = frequent_rdd.max(lambda x: x[1])
        most_freq_itemset_names = [id_to_skill[key] for key in most_freq_itemset[0]]
        print("The most frequent itemset of skills:", ", ".join(most_freq_itemset_names))
        print("The most frequent itemset support value:", most_freq_itemset[1])
        print()

In [ ]:
apriori_algorithm(hashed_baskets, sample_threshold, id_to_skill)

Finding frequent singletons ...
The amount of frequent singletons: 164
The most frequent singleton skill: Communication
The most frequent singleton support value: 714

Finding frequent pairs ...
The amount of frequent pairs: 172
The most frequent itemset of skills: Teamwork, Communication
The most frequent itemset support value: 252

Finding frequent triplets ...
The amount of frequent triplets: 70
The most frequent itemset of skills: Teamwork, Communication, Leadership
The most frequent itemset support value: 96

Finding frequent itemsets of size 4 ...
The amount of frequent itemsets of size 4 : 9
The most frequent itemset of skills: Teamwork, Problemsolving, Communication, Customer service
The most frequent itemset support value: 48

Finding frequent itemsets of size 5 ...
There are no frequent itemsets of the size 5
According to the monotonicity assumption there are no frequent itemsets of the bigger size



### **Apriori Algorithm without main Soft Skills**

In [ ]:
soft_skills_to_remove = ['Communication', 'Teamwork', 'Leadership', 'Problemsolving', 'Customer service']
skills_to_remove_id = [skill_to_id[skill] for skill in soft_skills_to_remove]

hashed_baskets_no_soft_skills = hashed_baskets.map(lambda basket: [x for x in basket if x not in skills_to_remove_id])

In [ ]:
apriori_algorithm(hashed_baskets_no_soft_skills, sample_threshold, id_to_skill)

Finding frequent singletons ...
The amount of frequent singletons: 159
The most frequent singleton skill: Customer Service
The most frequent singleton support value: 223

Finding frequent pairs ...
The amount of frequent pairs: 48
The most frequent itemset of skills: Customer Service, Problem Solving
The most frequent itemset support value: 76

Finding frequent triplets ...
The amount of frequent triplets: 1
The most frequent itemset of skills: Customer Service, Time Management, Problem Solving
The most frequent itemset support value: 32

Finding frequent itemsets of size 4 ...
There are no frequent itemsets of the size 4
According to the monotonicity assumption there are no frequent itemsets of the bigger size



In [ ]:
soft_skills_to_remove_v2 = ['Customer Service', 'Time Management', 'Problem Solving']
skills_to_remove_id_v2 = [skill_to_id[skill] for skill in soft_skills_to_remove_v2]

hashed_baskets_no_soft_skills_v2 = hashed_baskets_no_soft_skills.map(lambda basket: [x for x in basket if x not in skills_to_remove_id_v2])

In [ ]:
apriori_algorithm(hashed_baskets_no_soft_skills_v2, sample_threshold, id_to_skill)

Finding frequent singletons ...
The amount of frequent singletons: 156
The most frequent singleton skill: Communication skills
The most frequent singleton support value: 214

Finding frequent pairs ...
The amount of frequent pairs: 30
The most frequent itemset of skills: Attention to detail, Time management
The most frequent itemset support value: 50

Finding frequent triplets ...
There are no frequent itemsets of the size 3
According to the monotonicity assumption there are no frequent itemsets of the bigger size



In [ ]:
soft_skills_to_remove_v3 = ['Communication skills', 'Attention to Detail', 'Time management']
skills_to_remove_id_v3 = [skill_to_id[skill] for skill in soft_skills_to_remove_v3]

hashed_baskets_no_soft_skills_v3 = hashed_baskets_no_soft_skills_v2.map(lambda basket: [x for x in basket if x not in skills_to_remove_id_v3])

In [ ]:
apriori_algorithm(hashed_baskets_no_soft_skills_v3, sample_threshold, id_to_skill)

Finding frequent singletons ...
The amount of frequent singletons: 153
The most frequent singleton skill: Sales
The most frequent singleton support value: 189

Finding frequent pairs ...
The amount of frequent pairs: 22
The most frequent itemset of skills: Patient Care, Nursing
The most frequent itemset support value: 49

Finding frequent triplets ...
There are no frequent itemsets of the size 3
According to the monotonicity assumption there are no frequent itemsets of the bigger size



In [ ]:
no_soft_skills_threshold = round(sample_threshold * 0.7)

In [ ]:
apriori_algorithm(hashed_baskets_no_soft_skills_v3, no_soft_skills_threshold, id_to_skill)

Finding frequent singletons ...
The amount of frequent singletons: 234
The most frequent singleton skill: Sales
The most frequent singleton support value: 189

Finding frequent pairs ...
The amount of frequent pairs: 73
The most frequent itemset of skills: Patient Care, Nursing
The most frequent itemset support value: 49

Finding frequent triplets ...
The amount of frequent triplets: 6
The most frequent itemset of skills: Walking, Lifting, Standing
The most frequent itemset support value: 26

Finding frequent itemsets of size 4 ...
There are no frequent itemsets of the size 4
According to the monotonicity assumption there are no frequent itemsets of the bigger size

